In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import time

In [2]:
df = pd.read_csv("gidcc_daily_report_list.csv")

In [3]:
# df = df.head()

In [4]:
url = "http://www.gidcc.or.kr/19316/"
def get_page_html(url):
    response = requests.get(url)
    html = bs(response.text)
    time.sleep(0.2)
    return html

# get_page_html(url)

In [5]:
response = requests.get(url)
html = bs(response.text)

In [6]:
# body > div.g-container-sub > div > div.g-content.gnews-view > article > div.article-body
html.select("div.article-body > p")[0].get_text()

'경기도 일일상황분석 보고서 185호'

In [7]:
html.select("div.article-body > p > a")[0]["href"]

'http://www.gidcc.or.kr/wp-content/uploads/2020/09/경기도-일일상황분석-보고서-185호2020.9.22..pdf'

In [8]:
html.select("div.article-body > ul")[0].get_text()

'\nCOVID-19 맥락에서 지역 사회 내 아동의 마스크 사용에 대한 권고(WHO, 20.08.21)\n'

In [9]:
def parse_html(html):
    page_info = []
    file = html.select("div.article-body > p > a")[0]["href"]
    text = html.select("div.article-body")[0].get_text()
    page_info.append(file)
    page_info.append(text)
    return page_info

In [10]:
parse_html(html)[1]

'\n경기도 일일상황분석 보고서 185호\n\nCOVID-19 맥락에서 지역 사회 내 아동의 마스크 사용에 대한 권고(WHO, 20.08.21)\n\n'

In [11]:
def get_detail(url):
    html = get_page_html(url)
    page_info = parse_html(html)
    return page_info

In [12]:
from tqdm import tqdm

tqdm.pandas()

/Users/corazzon/opt/anaconda3/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [13]:
df["detail"] = df["url"].progress_apply(get_detail)

100%|██████████| 186/186 [01:51<00:00,  1.67it/s]


In [14]:
# df[] df["detail"].map(lambda x : x[0])

In [18]:
df["file"] = df["detail"].map(lambda x : x[0])
df["desc"] = df["detail"].map(lambda x : x[1])

In [23]:
df["title"] = df["desc"].str.split("\n", expand=True)[1]

In [27]:
df["desc_text"] = df["desc"].str.replace("\n경기도 일일상황분석 보고서", "")

In [31]:
df_result = df[["url", "file", "title", "desc_text"]]

In [32]:
df_result.to_csv("gidcc_daily_report_detail.csv", index=False)